In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.lite as tflite

from tensorflow import keras

2023-11-20 19:38:12.902715: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 19:38:12.902752: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 19:38:12.902823: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 19:38:12.911416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Homework

In this homework, we'll deploy the bees vs wasps model we trained in the 
[previous homework](../08-deep-learning/homework.md).

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5



## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb

- 43 Mb (44 but 43 is the nearest)

In [4]:
model = keras.models.load_model('bees-wasps.h5')

In [5]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('bees-wasps-model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp69_9dthd/assets


INFO:tensorflow:Assets written to: /tmp/tmp69_9dthd/assets
2023-11-20 19:39:03.241526: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-20 19:39:03.241551: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-20 19:39:03.241764: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp69_9dthd
2023-11-20 19:39:03.242572: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-20 19:39:03.242586: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp69_9dthd
2023-11-20 19:39:03.245132: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-20 19:39:03.334934: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp69_9dthd
2023-11-20 19:39:03.345249: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [6]:
interpreter = tflite.Interpreter(model_path='bees-wasps-model.tflite')
interpreter.allocate_tensors()

input_tensor_index = interpreter.get_input_details()[0]['index']
output_tensor_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Question 2

In [3]:
output_tensor_index

13

In [7]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img


## Question 3

In [8]:
image = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')
img = prepare_image(image, (150,150))

In [9]:
x = np.array(img, dtype='float32')
X = 1./255 * np.array([x])
X[0,0,0,0]

0.9450981

## Question 4

In [10]:
interpreter.set_tensor(input_tensor_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_tensor_index)
preds

array([[0.65898407]], dtype=float32)



## Question 5




- 662 Mb


## Question 6


- 0.4453